In [24]:
import qualified Data.ByteString as B
import qualified Data.Text as T
import qualified Data.Text.Encoding as TE
import Data.Word
import Data.Map (Map, (!))
import qualified Data.Map as Map
import Data.List
import Data.Ord
import Data.ByteString.Base64 (decodeBase64)

In [25]:
:load src/MinBPE.hs

In [70]:
import qualified Data.Attoparsec.ByteString.Char8 as Atto
import Control.Applicative ((<|>))

In [27]:
base64 :: Atto.Parser B.ByteString
base64 = do
    raw <- Atto.takeWhile1 (\c -> Atto.isAlpha_ascii c || Atto.isDigit c || Atto.inClass "+/=" c)
    case decodeBase64 raw of
        Left err -> fail $ "Error decoding Base64: " ++ show err
        Right bs -> return bs

In [28]:
parseEntry :: Atto.Parser (Token, B.ByteString)
parseEntry = do
    bs <- base64
    Atto.space
    t <- Atto.decimal
    return (t, bs)

In [95]:
toVocab :: [(Token, B.ByteString)] -> Vocab
toVocab = Map.fromList

In [96]:
parseFile :: Atto.Parser Vocab
parseFile = fmap toVocab $ Atto.many1 $ parseEntry <* Atto.endOfLine

In [103]:
tik <- B.readFile "example/cl100k_base.tiktoken"

In [97]:
foo = Atto.parseOnly parseFile $ B.take 100 tik

In [98]:
foo

Right (fromList [(0,"!"),(1,"\""),(2,"#"),(3,"$"),(4,"%"),(5,"&"),(6,"'"),(7,"("),(8,")"),(9,"*"),(10,"+"),(11,","),(12,"-")])

In [86]:
case foo of
    Atto.Fail _ _ _ -> print "error"
    Atto.Partial _ -> print "partial"
    Atto.Done _ r -> print "done"

Line 2: Use record patterns
Found:
Atto.Fail _ _ _
Why not:
Atto.Fail {}

"partial"

In [87]:
:t B.readFile

B.readFile :: FilePath -> IO ByteString

In [102]:
import System.Directory
getCurrentDirectory

"/Users/fnoble/projects/ML/minbpe-hs"